In [1]:
# Imports
#---------
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter, MonthLocator, DayLocator
import matplotlib as mpl
import warnings
import seaborn as sns
from scipy import stats
import copy
from scipy.stats import wilcoxon
import seaborn as sns
from matplotlib.patches import Rectangle

warnings.filterwarnings("ignore")

register_matplotlib_converters()

# File locations
#----------------
#the_gases = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/raw/1oxaria/json/gap_filling/'
pngs = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/gap_filling/536_method/lockdown_stats/pngs/'
data_home = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/raw/0oxaria/gap_filling/jun_to_sept_2021/'
stats_home = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/gap_filling/536_method/lockdown_stats/'
aurn_path = '/home/tonyb/Gdrive/MinicondaProjects/oxaria/data/aurn/aurn_2016_2021/'


cmap = copy.copy(plt.cm.get_cmap("Blues"))
cmap.set_under("white")

def color_nan_white(val):
#     """Color the nan text white"""
    if np.isnan(val):
        return 'color: white'

def color_nan_white_background(val):
#      """Color the nan cell background white"""
    if np.isnan(val):
        return 'background-color: white'
    
def color_nonnan_black(val):
#     """Color the non-nan text white"""
    if pd.notna(val):
        return 'color: black'
    
def _color_red_or_green(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

In [2]:
# Load senssor data at 24hr resolution
#--------------------------------------
sensors_24hr = pd.read_feather(stats_home+'sensors_24hr_9095.ftr').drop('index', axis=1)
sensors_24hr['fname'] = sensors_24hr['fname'].replace(regex={'Jahlul':'Ahlul', 'Mosque':'Centre','Marsten':'Marston'}) 


# Load sebbes typical values 2016-21 for graphing
#-------------------------------------------------
sebbes_pm10_ld_event_desc_mean = pd.read_feather(stats_home+'sebbes_ordinal_daymean_codeup_ak_mean_desc_pm10_1619_9095.ftr')
sebbes_pm25_ld_event_desc_mean = pd.read_feather(stats_home+'sebbes_ordinal_daymean_codeup_ak_mean_desc_pm25_1619_9095.ftr')

# Load sebbes data with ordinal day & LD event codes
#-----------------------------------------------------
sebbes_ordinal_daymean_codeup_ak = pd.read_feather(stats_home+'sebbes_ordinal_daymean_codeup_ak_1619_9095.ftr')

In [3]:
# Fill gaps in data with NaNs for easy plotting - pm10
#------------------------------------------------------

sensor_ld_event_stats = sensors_24hr.groupby(['fname','ld_code']).mean().sort_values(['fname','ld_code'])
sensors = pd.DataFrame(sorted(list(sensor_ld_event_stats.index.get_level_values(0).unique())), columns=['sensor'])
events = pd.DataFrame(sorted(['a','b','c','d','e','f','g','h','i','j','k']))

tmplist = []
for i, j in sensors.iterrows():
    for ii, jj in events.iterrows():
        tmp = pd.DataFrame(zip(j,jj), columns=['sensors','events'])
        tmplist.append(tmp)
sevents = pd.concat(tmplist).reset_index(drop=True)

sensor_ld_event_stats_1 = sevents.merge(sensor_ld_event_stats.reset_index(), left_on=['sensors','events'], right_on=['fname','ld_code'], how='left')
sensor_ld_event_stats_1['ld_code'] = sensor_ld_event_stats_1['events']
sensor_ld_event_stats_1['fname'] = sensor_ld_event_stats_1['sensors']
sensor_ld_event_stats_1.drop(['sensors','events'], inplace=True, axis=1)
sensor_ld_event_stats_1.set_index('ld_code', inplace=True)

sensors_24hr['fname'].unique()

array(['Ahlul Bayt Centre', 'County Hall', 'Divinity Rd', 'High St',
       'Jesus College', 'John Radcliffe', 'New Marston',
       'Said Business School', 'South Parks Rd', 'Speedwell St',
       'St Ebbes', 'St Giles', 'The Plain', 'Warneford Hospital',
       'Worcester College'], dtype=object)

In [4]:
# # Creat a table of differentials - pm10 mean
# #---------------------------------------------
# # df = sensor_ld_event_stats_1.iloc[:,[0,1,2,3,4,5,6,7,8]].reset_index().merge(sebbes_pm10_ld_event_desc_mean[['ld_code','pm10_mean','pm10_95%']], 
# #                                                              left_on='ld_code',
# #                                                              right_on='ld_code',
# #                                                              how='left',
# #                                                              suffixes=(None,'_s')
# #                                                             ).rename(columns={'pm10_95%':'pm10_p95_s'})

# df = sensor_ld_event_stats_1.reset_index().merge(sebbes_pm10_ld_event_desc_mean[['ld_code','pm10_mean','pm10_90%','pm10_95%']], 
#                                                              left_on='ld_code',
#                                                              right_on='ld_code',
#                                                              how='left',
#                                                              suffixes=(None,'_s')
#                                                             ).rename(columns={'pm10_95%':'pm10_p95_s','pm10_90%':'pm10_p90_s'})

# df['dif_pm10_mean'] = df['pm10_mean'] - df['pm10_mean_s']
# df['dif_pm10_95'] = df['pm10_p95'] - df['pm10_p95_s']
# df['dif_pm10_90'] = df['pm10_p90'] - df['pm10_p90_s']

# df = df.rename(columns={'ld_code': 'Event code', 'fname': 'Sensor'})
# dif_pm10_mean = df[['Event code','Sensor','dif_pm10_mean']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
# dif_pm10_mean.loc['Mean',:] = dif_pm10_mean.mean(axis=0)
# dif_pm10_mean.loc[:,'Mean'] = dif_pm10_mean.mean(axis=1)


# tmplist = []
# for i, g in sensor_sebbes.groupby(['fname','ld_code']):
#     stat, p = wilcoxon(g['pm10_mean'].values, g['pm10_mean_s'].values) 
#     tmplist.append([i[0],i[1],stat,p])
# pm10_mean_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

# dif_pm10_p95 = df[['Event code','Sensor','dif_pm10_95']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
# dif_pm10_p95.loc['Mean',:] = dif_pm10_p95.mean(axis=0)
# dif_pm10_p95.loc[:,'Mean'] = dif_pm10_p95.mean(axis=1)

# tmplist = []
# for i, g in sensor_sebbes.groupby(['fname','ld_code']):
#     stat, p = wilcoxon(g['pm10_p95'].values, g['pm10_p95_s'].values) 
#     tmplist.append([i[0],i[1],stat,p])
# pm10_p95_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

# dif_pm10_p90 = df[['Event code','Sensor','dif_pm10_90']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
# dif_pm10_p90.loc['Mean',:] = dif_pm10_p90.mean(axis=0)
# dif_pm10_p90.loc[:,'Mean'] = dif_pm10_p90.mean(axis=1)

# tmplist = []
# for i, g in sensor_sebbes.groupby(['fname','ld_code']):
#     stat, p = wilcoxon(g['pm10_p90'].values, g['pm10_p90_s'].values) 
#     tmplist.append([i[0],i[1],stat,p])
# pm10_p90_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

# pm10_mean_wilcoxon_a = pm10_mean_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
# pm10_mean_wilcoxon_a.loc['Mean',:] = pm10_mean_wilcoxon_a.mean(axis=0)
# pm10_mean_wilcoxon_a.loc[:,'Mean'] = pm10_mean_wilcoxon_a.mean(axis=1)

# pm10_p95_wilcoxon_a = pm10_p95_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
# pm10_p95_wilcoxon_a.loc['Mean',:] = pm10_p95_wilcoxon_a.mean(axis=0)
# pm10_p95_wilcoxon_a.loc[:,'Mean'] = pm10_p95_wilcoxon_a.mean(axis=1)

# # pm10_p90_wilcoxon_a = pm10_p90_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
# pm10_p90_wilcoxon_a.loc['Mean',:] = pm10_p90_wilcoxon_a.mean(axis=0)
# pm10_p90_wilcoxon_a.loc[:,'Mean'] = pm10_p90_wilcoxon_a.mean(axis=1)

# display(dif_pm10_mean.style
#     .background_gradient(axis=None)
#     .applymap(lambda x: color_nan_white_background(x))
#     .applymap(lambda x: color_nonnan_black(x))
#     .applymap(lambda x: color_nan_white(x))
#          .apply(lambda x: np.where(
#     pm10_mean_wilcoxon_a < 0.05,               # Compare DataFrames
#     'color: red',  # True Styles
#     ''                       # False Styles
# ), axis=None).apply(lambda x: np.where(
#     pm10_mean_wilcoxon_a < 0.05,               # Compare DataFrames
#     'font-weight: bold',  # True Styles
#     ''                       # False Styles
# ), axis=None).format("{:.1f}"),
#         dif_pm10_p95.style
#     .background_gradient(axis=None)
#     .applymap(lambda x: color_nan_white_background(x))
#     .applymap(lambda x: color_nonnan_black(x))
#     .applymap(lambda x: color_nan_white(x))
#          .apply(lambda x: np.where(
#     pm10_p95_wilcoxon_a < 0.05,               # Compare DataFrames
#     'color: red',  # True Styles
#     ''                       # False Styles
# ), axis=None).apply(lambda x: np.where(
#     pm10_p95_wilcoxon_a < 0.05,               # Compare DataFrames
#     'font-weight: bold',  # True Styles
#     ''                       # False Styles
# ), axis=None).format("{:.1f}"),
#         dif_pm10_p95.style
#     .background_gradient(axis=None)
#     .applymap(lambda x: color_nan_white_background(x))
#     .applymap(lambda x: color_nonnan_black(x))
#     .applymap(lambda x: color_nan_white(x))
#          .apply(lambda x: np.where(
#     pm10_p95_wilcoxon_a < 0.05,               # Compare DataFrames
#     'color: red',  # True Styles
#     ''                       # False Styles
# ), axis=None).apply(lambda x: np.where(
#     pm10_p95_wilcoxon_a < 0.05,               # Compare DataFrames
#     'font-weight: bold',  # True Styles
#     ''                       # False Styles
# ), axis=None).format("{:.1f}")
        
       
#        )


NameError: name 'sensor_sebbes' is not defined

In [10]:
df = sensor_ld_event_stats_1.reset_index().merge(sebbes_pm25_ld_event_desc_mean[['ld_code','pm25_mean','pm25_90%','pm25_95%']], 
                                                             left_on='ld_code',
                                                             right_on='ld_code',
                                                             how='left',
                                                             suffixes=(None,'_s')
                                                            ).rename(columns={'pm25_95%':'pm25_p95_s','pm25_90%':'pm25_p90_s'})

df['dif_pm25_mean'] = df['pm25_mean'] - df['pm25_mean_s']
df['dif_pm25_95'] = df['pm25_p95'] - df['pm25_p95_s']
df['dif_pm25_90'] = df['pm25_p90'] - df['pm25_p90_s']


df = df.rename(columns={'ld_code': 'Event code', 'fname': 'Sensor'})
dif_pm25_mean = df[['Event code','Sensor','dif_pm25_mean']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
dif_pm25_mean.loc['Mean',:] = dif_pm25_mean.mean(axis=0)
dif_pm25_mean


Event code,a,b,c,d,e,f,g,h,i,j,k
Sensor,,,,,,,,,,,
Ahlul Bayt Centre,NaN,NaN,-0.400000,-1.800000,1.600000,0.3,-2.700000,-3.600000,-2.400000,-3.000000,-0.1
County Hall,NaN,NaN,NaN,NaN,NaN,0.1,-1.800000,-2.500000,-1.600000,-2.200000,1.6
Divinity Rd,NaN,NaN,2.800000,2.100000,5.100000,5.5,1.100000,1.500000,1.200000,-0.000000,2.6
High St,NaN,NaN,NaN,5.200000,6.400000,5.5,1.600000,3.800000,1.300000,-2.200000,2.2
Jesus College,-2.000000,-0.700,-1.300000,-0.800000,3.100000,3.6,0.500000,0.300000,-0.200000,-2.800000,0.7
John Radcliffe,NaN,NaN,NaN,NaN,NaN,2.1,-1.800000,-1.900000,-2.300000,-3.400000,-0.0
New Marston,NaN,NaN,-0.200000,-1.900000,3.200000,0.5,-4.100000,-4.300000,-3.800000,-4.000000,NaN
Said Business School,NaN,NaN,-0.500000,-2.500000,2.300000,0.4,-2.800000,0.500000,-3.100000,-2.800000,-0.5
South Parks Rd,NaN,NaN,1.100000,-1.900000,2.200000,3.6,-0.500000,-0.500000,-0.400000,0.000000,3.9


In [6]:
# Creat a table of differentials - pm2.5 mean
#---------------------------------------------
df = sensor_ld_event_stats_1.reset_index().merge(sebbes_pm25_ld_event_desc_mean[['ld_code','pm25_mean','pm25_90%','pm25_95%']], 
                                                             left_on='ld_code',
                                                             right_on='ld_code',
                                                             how='left',
                                                             suffixes=(None,'_s')
                                                            ).rename(columns={'pm25_95%':'pm25_p95_s','pm25_90%':'pm25_p90_s'})

df['dif_pm25_mean'] = df['pm25_mean'] - df['pm25_mean_s']
df['dif_pm25_95'] = df['pm25_p95'] - df['pm25_p95_s']
df['dif_pm25_90'] = df['pm25_p90'] - df['pm25_p90_s']


df = df.rename(columns={'ld_code': 'Event code', 'fname': 'Sensor'})
dif_pm25_mean = df[['Event code','Sensor','dif_pm25_mean']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
dif_pm25_mean.loc['Mean',:] = dif_pm25_mean.mean(axis=0)
dif_pm25_mean.loc[:,'Mean'] = dif_pm25_mean.mean(axis=1)

tmplist = []
for i, g in sensor_sebbes.groupby(['fname','ld_code']):
    stat, p = wilcoxon(x=g['pm25_mean'].values, y=g['pm25_mean_s'].values) 
    tmplist.append([i[0],i[1],stat,p])
pm25_mean_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

dif_pm25_p95 = df[['Event code','Sensor','dif_pm25_95']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
dif_pm25_p95.loc['Mean',:] = dif_pm25_p95.mean(axis=0)
dif_pm25_p95.loc[:,'Mean'] = dif_pm25_p95.mean(axis=1)

tmplist = []
for i, g in sensor_sebbes.groupby(['fname','ld_code']):
    stat, p = wilcoxon(x=g['pm25_p95'].values, y=g['pm25_p95_s'].values) 
    tmplist.append([i[0],i[1],stat,p])
pm25_p95_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

dif_pm25_p90 = df[['Event code','Sensor','dif_pm25_90']].pivot(index='Sensor',columns='Event code').round(1).droplevel(0, axis=1)
dif_pm25_p90.loc['Mean',:] = dif_pm25_p90.mean(axis=0)
dif_pm25_p90.loc[:,'Mean'] = dif_pm25_p90.mean(axis=1)

tmplist = []
for i, g in sensor_sebbes.groupby(['fname','ld_code']):
    stat, p = wilcoxon(x=g['pm25_p90'].values, y=g['pm25_p90_s'].values) 
    tmplist.append([i[0],i[1],stat,p])
pm25_p90_wilcoxon = pd.DataFrame(tmplist, columns = ['fname','ld_code', 'stat', 'P'])

tmpdfmean = pm25_mean_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
tmpdfmean.loc['Mean',:] = tmpdfmean.mean(axis=0)
tmpdfmean.loc[:,'Mean'] = tmpdfmean.mean(axis=1)

tmpdfp95 = pm25_p95_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
tmpdfp95.loc['Mean',:] = tmpdfp95.mean(axis=0)
tmpdfp95.loc[:,'Mean'] = tmpdfp95.mean(axis=1)

tmpdfp90 = pm25_p90_wilcoxon[['fname','ld_code', 'P']].pivot(index='fname',columns='ld_code').droplevel(0, axis=1)
tmpdfp90.loc['Mean',:] = tmpdfp90.mean(axis=0)
tmpdfp90.loc[:,'Mean'] = tmpdfp90.mean(axis=1)

display(dif_pm25_mean.style
    .background_gradient(axis=None)
    .applymap(lambda x: color_nan_white_background(x))
    .applymap(lambda x: color_nonnan_black(x))
    .applymap(lambda x: color_nan_white(x))
         .apply(lambda x: np.where(
    tmpdfmean < 0.05,               # Compare DataFrames
    'color: red',  # True Styles
    ''                       # False Styles
), axis=None).apply(lambda x: np.where(
    tmpdfmean < 0.05,               # Compare DataFrames
    'font-weight: bold',  # True Styles
    ''                       # False Styles
), axis=None).format("{:.1f}"),
        dif_pm25_p95.style
    .background_gradient(axis=None)
    .applymap(lambda x: color_nan_white_background(x))
    .applymap(lambda x: color_nonnan_black(x))
    .applymap(lambda x: color_nan_white(x))
         .apply(lambda x: np.where(
    tmpdfmean < 0.05,               # Compare DataFrames
    'color: red',  # True Styles
    ''                       # False Styles
), axis=None).apply(lambda x: np.where(
    tmpdfp95 < 0.05,               # Compare DataFrames
    'font-weight: bold',  # True Styles
    ''                       # False Styles
), axis=None).format("{:.1f}"),
        dif_pm25_p90.style
    .background_gradient(axis=None)
    .applymap(lambda x: color_nan_white_background(x))
    .applymap(lambda x: color_nonnan_black(x))
    .applymap(lambda x: color_nan_white(x))
         .apply(lambda x: np.where(
    tmpdfmean < 0.05,               # Compare DataFrames
    'color: red',  # True Styles
    ''                       # False Styles
), axis=None).apply(lambda x: np.where(
    tmpdfp90 < 0.05,               # Compare DataFrames
    'font-weight: bold',  # True Styles
    ''                       # False Styles
), axis=None).format("{:.1f}")
       )


NameError: name 'sensor_sebbes' is not defined

In [7]:
# Heat map - pm10 & pm2.5 daily mean
#------------------------------------
sort_map = {'County Hall':5,
            'Divinity Rd':0,
            'High St':1,
            'Ahlul Bayt Centre':4,
            'Jesus College':6,
            'John Radcliffe':7,
            'New Marston':13,
            'Said Business School':8,
            'South Parks Rd':2,
            'Speedwell St':9,
            'St Ebbes':14,
            'St Giles':10,
            'The Plain':11,
            'Warneford Hospital':12,
            'Worcester College':3,
            'Mean':15
           }

dif_pm10_mean['order'] = dif_pm10_mean.index.map(sort_map)
dif_pm10_mean1 = dif_pm10_mean[:-1]
dif_pm10_mean1['order'] = dif_pm10_mean1['order'].astype('int32')
dif_pm10_mean1.sort_values(by='order',inplace=True)
dif_pm10_mean1 = dif_pm10_mean1.drop(columns=['order'])

dif_pm25_mean['order'] = dif_pm25_mean.index.map(sort_map)
dif_pm25_mean1 = dif_pm25_mean[:-1]
dif_pm25_mean1['order'] = dif_pm25_mean1['order'].astype('int32')
dif_pm25_mean1.sort_values(by='order',inplace=True)
dif_pm25_mean1 = dif_pm25_mean1.drop(columns=['order'])

pm10_mean_wilcoxon['order'] = pm10_mean_wilcoxon['fname'].map(sort_map)
pm10_mean_wilcoxon['order'] = pm10_mean_wilcoxon['order'].astype('int32')
pm10_mean_wilcoxon.sort_values(by='order',inplace=True)

pm25_mean_wilcoxon['order'] = pm25_mean_wilcoxon['fname'].map(sort_map)
pm25_mean_wilcoxon['order'] = pm25_mean_wilcoxon['order'].astype('int32')
pm25_mean_wilcoxon.sort_values(by='order',inplace=True)


fig, (ax1, ax2)  = plt.subplots(1,2, figsize=(17,9))

sns.heatmap(dif_pm10_mean1, annot=True, cmap='coolwarm', robust=True, ax=ax1, linewidths=.5, vmin=-8, vmax=8, cbar_kws={'label': 'Change (ug/m3)'})
tmpdf = pm10_mean_wilcoxon[['order','ld_code', 'P'
                          ]].pivot(index='order',columns='ld_code').droplevel([0], axis=1
                                            ).reset_index(drop=True
                                                         ).T.reset_index(drop=True
                                                                        ).T.stack()
tmpdf = tmpdf[tmpdf > 0.05]
t = tmpdf.index.values.tolist()
for i in t:
    ax1.add_patch(Rectangle((i[1], i[0]), 1, 1, fill=False, edgecolor='black', lw=2, hatch='//'))


# pm2.5
#-------
sns.heatmap(dif_pm25_mean1, annot=True, cmap='coolwarm', robust=True, ax=ax2, linewidths=.5, vmin=-8, vmax=8, cbar_kws={'label': 'Change (ug/m3)'})
tmpdf = pm25_mean_wilcoxon[['order','ld_code', 'P'
                          ]].pivot(index='order',columns='ld_code').droplevel([0], axis=1
                                            ).reset_index(drop=True
                                                         ).T.reset_index(drop=True
                                                                        ).T.stack()
tmpdf = tmpdf[tmpdf > 0.05]
t = tmpdf.index.values.tolist()
for i in t:
    ax2.add_patch(Rectangle((i[1], i[0]), 1, 1, fill=False, edgecolor='black', lw=2, hatch='//'))


    
ax1.set_title('(a)   Daily mean PM10 by lockdown event', y=1.01, x=0.4, fontsize=14.5)    
ax2.set_title('(b)   Daily mean PM2.5 by lockdown event', y=1.01, x=0.4, fontsize=14.5)
ax1.set_xlabel('Lockdown event code', fontsize=13)
ax1.set_ylabel('Sensor location', fontsize=13)
ax2.set_xlabel('Lockdown event code', fontsize=13)
ax2.set_ylabel('Sensor location', fontsize=13)

fig.suptitle('Measured differentials in PM10 & PM2.5 concentration metrics during 2020-21 COVID-19 pandemic restriction events relative to typical levels experienced 2016-21 in Oxford, UK.\nDifferences shown in mass units (ug/m3). All changes statistically significant differences unless denoted by shading.', fontsize=14, y=1)

#plt.tight_layout() 
plt.subplots_adjust(wspace=0.35, hspace=0.2)

#plt.savefig('sensor_vs_typical_year_heatmap_24hrmean_1619.png')

plt.show()

    

NameError: name 'pm10_mean_wilcoxon' is not defined

In [8]:
# Calculate the differential in PM10 to PM2.5 ratio
#---------------------------------------------------
dfratio = sensor_ld_event_stats_1.reset_index().merge(sebbes_pm10_ld_event_desc_mean[['ld_code','pm10_mean','pm10_90%','pm10_95%']], 
                                                             left_on='ld_code',
                                                             right_on='ld_code',
                                                             how='left',
                                                             suffixes=(None,'_s')
                                                            ).rename(columns={'pm10_95%':'pm10_p95_s','pm10_90%':'pm10_p90_s'})

dfratio1 = dfratio.reset_index().merge(sebbes_pm25_ld_event_desc_mean[['ld_code','pm25_mean','pm25_90%','pm25_95%']], 
                                                             left_on='ld_code',
                                                             right_on='ld_code',
                                                             how='left',
                                                             suffixes=(None,'_s')
                                                            ).rename(columns={'pm25_95%':'pm25_p95_s','pm25_90%':'pm25_p90_s'})

dfratio1['sensor_pm10225_ratio'] = dfratio1['pm25_mean'] / dfratio1['pm10_mean']
dfratio1['sebbes_pm10225_ratio'] = dfratio1['pm25_mean_s'] / dfratio1['pm10_mean_s']
dfratio1['dif_pm10225_ratio'] = dfratio1['sensor_pm10225_ratio'] - dfratio1['sebbes_pm10225_ratio']

dfratio1 = dfratio1.rename(columns={'ld_code': 'Event code', 'fname': 'Sensor'})
dif_dfratio1 = dfratio1[['Event code','Sensor','sensor_pm10225_ratio']].pivot(index='Sensor',columns='Event code').droplevel(0, axis=1)
dif_dfratio1.loc['Mean',:] = dif_dfratio1.mean(axis=0)
dif_dfratio1.loc[:,'Mean'] = dif_dfratio1.mean(axis=1)
dif_dfratio1

Event code,a,b,c,d,e,f,g,h,i,j,k,Mean
Sensor,,,,,,,,,,,,
Ahlul Bayt Centre,NaN,NaN,0.645467,0.750594,0.964230,0.907080,0.961567,0.918494,0.884454,0.836218,0.769215,0.848591
County Hall,NaN,NaN,NaN,NaN,NaN,0.907238,0.964253,0.953771,0.872713,0.819623,0.797349,0.885824
Divinity Rd,NaN,NaN,0.592053,0.691063,0.839848,0.930095,0.842543,0.751057,0.738556,0.667532,0.665561,0.746479
High St,NaN,NaN,NaN,0.745046,0.793766,0.919763,0.838833,0.747370,0.668289,0.566992,0.642623,0.740335
Jesus College,0.708581,0.732202,0.626143,0.680372,0.809099,0.792267,0.790153,0.766786,0.610647,0.676237,0.772395,0.724080
John Radcliffe,NaN,NaN,NaN,NaN,NaN,0.763753,0.808919,0.690403,0.625035,0.509233,0.587016,0.664060
New Marston,NaN,NaN,0.622611,0.606669,0.808922,0.816899,0.915683,0.924482,0.856349,0.809978,NaN,0.795199
Said Business School,NaN,NaN,0.647588,0.665596,0.977989,0.926032,0.946448,1.044556,0.802452,0.759310,0.712980,0.831439
South Parks Rd,NaN,NaN,0.565735,0.594352,0.876873,0.876233,0.846483,0.749038,0.692100,0.614434,0.646783,0.718003


In [ ]:
dfratio1

,index,Event code,Sensor,pm10_count,pm10_mean,pm10_std,pm10_min,pm10_p50,pm10_p90,pm10_p95,...,ordinal_days,pm10_mean_s,pm10_p90_s,pm10_p95_s,pm25_mean_s,pm25_p90_s,pm25_p95_s,sensor_pm10225_ratio,sebbes_pm10225_ratio,dif_pm10225_ratio
76,76,k,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.293480,16.887686,19.284143,6.965360,12.227996,14.170504,NaN,0.676677,NaN
67,67,b,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.633392,27.305294,30.854941,12.203299,21.877412,25.012412,NaN,0.780592,NaN
66,66,a,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.402617,28.707073,32.469756,12.349181,23.932593,27.024568,NaN,0.801759,NaN
68,68,c,New Marston,53.534483,10.915602,3.255004,5.643178,10.571693,14.936137,16.055396,...,239.500000,10.106749,16.273267,18.221411,6.994023,11.900386,13.549355,0.622611,0.692015,-0.069404
69,69,d,New Marston,57.146341,10.358538,3.414074,4.768032,10.033175,14.618328,15.799578,...,289.000000,11.792086,21.039329,25.052530,8.202249,16.217410,19.587518,0.606669,0.695572,-0.088904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,b,Worcester College,22.388235,19.047143,4.395377,13.292346,18.315581,23.993919,25.189982,...,125.000000,15.633392,27.305294,30.854941,12.203299,21.877412,25.012412,0.692612,0.780592,-0.087980
161,161,h,Worcester College,39.314286,25.851408,7.048840,15.560341,24.615687,34.034911,37.730409,...,84.000000,17.025160,31.028000,35.336429,13.164310,25.050571,28.549429,0.708729,0.773227,-0.064498
154,154,a,Worcester College,60.717391,11.988682,4.565868,5.644186,11.020892,18.181956,20.105378,...,59.500000,15.402617,28.707073,32.469756,12.349181,23.932593,27.024568,0.762535,0.801759,-0.039223
156,156,c,Worcester College,46.445545,8.685592,2.794337,4.721471,8.286259,12.317024,13.243342,...,218.000000,10.106749,16.273267,18.221411,6.994023,11.900386,13.549355,0.788304,0.692015,0.096288


In [ ]:
dfratio1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 76 to 162
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 163 non-null    int64  
 1   Event code            163 non-null    object 
 2   Sensor                163 non-null    object 
 3   pm10_count            129 non-null    float64
 4   pm10_mean             129 non-null    float64
 5   pm10_std              129 non-null    float64
 6   pm10_min              129 non-null    float64
 7   pm10_p50              129 non-null    float64
 8   pm10_p90              129 non-null    float64
 9   pm10_p95              129 non-null    float64
 10  pm10_max              129 non-null    float64
 11  pm25_count            129 non-null    float64
 12  pm25_mean             129 non-null    float64
 13  pm25_std              129 non-null    float64
 14  pm25_min              129 non-null    float64
 15  pm25_p50              

In [ ]:
dfratio1

,index,Event code,Sensor,pm10_count,pm10_mean,pm10_std,pm10_min,pm10_p50,pm10_p90,pm10_p95,...,ordinal_days,pm10_mean_s,pm10_p90_s,pm10_p95_s,pm25_mean_s,pm25_p90_s,pm25_p95_s,sensor_pm10225_ratio,sebbes_pm10225_ratio,dif_pm10225_ratio
76,76,k,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.293480,16.887686,19.284143,6.965360,12.227996,14.170504,NaN,0.676677,NaN
67,67,b,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.633392,27.305294,30.854941,12.203299,21.877412,25.012412,NaN,0.780592,NaN
66,66,a,New Marston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.402617,28.707073,32.469756,12.349181,23.932593,27.024568,NaN,0.801759,NaN
68,68,c,New Marston,53.534483,10.915602,3.255004,5.643178,10.571693,14.936137,16.055396,...,239.500000,10.106749,16.273267,18.221411,6.994023,11.900386,13.549355,0.622611,0.692015,-0.069404
69,69,d,New Marston,57.146341,10.358538,3.414074,4.768032,10.033175,14.618328,15.799578,...,289.000000,11.792086,21.039329,25.052530,8.202249,16.217410,19.587518,0.606669,0.695572,-0.088904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,b,Worcester College,22.388235,19.047143,4.395377,13.292346,18.315581,23.993919,25.189982,...,125.000000,15.633392,27.305294,30.854941,12.203299,21.877412,25.012412,0.692612,0.780592,-0.087980
154,154,a,Worcester College,60.717391,11.988682,4.565868,5.644186,11.020892,18.181956,20.105378,...,59.500000,15.402617,28.707073,32.469756,12.349181,23.932593,27.024568,0.762535,0.801759,-0.039223
161,161,h,Worcester College,39.314286,25.851408,7.048840,15.560341,24.615687,34.034911,37.730409,...,84.000000,17.025160,31.028000,35.336429,13.164310,25.050571,28.549429,0.708729,0.773227,-0.064498
156,156,c,Worcester College,46.445545,8.685592,2.794337,4.721471,8.286259,12.317024,13.243342,...,218.000000,10.106749,16.273267,18.221411,6.994023,11.900386,13.549355,0.788304,0.692015,0.096288


In [ ]:

sort_map1 = {
'New Marston': 0,
'St Ebbes': 1,
'Jesus College': 2,
'Said Business School': 3,
'St Giles': 4,
'Ahlul Bayt Centre': 5,
'County Hall': 6,
'Divinity Rd': 7,
'High St': 8,
'John Radcliffe': 9,
'South Parks Rd': 10,
'Speedwell St': 11,
'The Plain': 12,
'Warneford Hospital': 13,
'Worcester College': 14,
'Mean': 15
}

dfratio1x = dfratio1[['Event code','Sensor','sensor_pm10225_ratio']].pivot(index='Sensor',columns='Event code').droplevel(0, axis=1)
dfratio1x

Event code,a,b,c,d,e,f,g,h,i,j,k
Sensor,,,,,,,,,,,
Ahlul Bayt Centre,NaN,NaN,0.645467,0.750594,0.964230,0.907080,0.961567,0.918494,0.884454,0.836218,0.769215
County Hall,NaN,NaN,NaN,NaN,NaN,0.907238,0.964253,0.953771,0.872713,0.819623,0.797349
Divinity Rd,NaN,NaN,0.592053,0.691063,0.839848,0.930095,0.842543,0.751057,0.738556,0.667532,0.665561
High St,NaN,NaN,NaN,0.745046,0.793766,0.919763,0.838833,0.747370,0.668289,0.566992,0.642623
Jesus College,0.708581,0.732202,0.626143,0.680372,0.809099,0.792267,0.790153,0.766786,0.610647,0.676237,0.772395
John Radcliffe,NaN,NaN,NaN,NaN,NaN,0.763753,0.808919,0.690403,0.625035,0.509233,0.587016
New Marston,NaN,NaN,0.622611,0.606669,0.808922,0.816899,0.915683,0.924482,0.856349,0.809978,NaN
Said Business School,NaN,NaN,0.647588,0.665596,0.977989,0.926032,0.946448,1.044556,0.802452,0.759310,0.712980
South Parks Rd,NaN,NaN,0.565735,0.594352,0.876873,0.876233,0.846483,0.749038,0.692100,0.614434,0.646783


In [ ]:
dfratio1x.loc['Mean',:] = dfratio1x.mean(axis=0)
dfratio1x.loc[:,'Mean'] = dfratio1x.mean(axis=1)
dfratio1x['order'] = dfratio1x.index.map(sort_map1)
dfratio1x['order'] = dfratio1x['order'].astype('int32')
dfratio1x.sort_values(by='order',inplace=True)
dfratio1x = dfratio1x.drop(columns=['order'])

dfratio1x

Event code,a,b,c,d,e,f,g,h,i,j,k,Mean
Sensor,,,,,,,,,,,,
New Marston,NaN,NaN,0.622611,0.606669,0.808922,0.816899,0.915683,0.924482,0.856349,0.809978,NaN,0.794109
St Ebbes,NaN,0.693279,0.569947,0.606018,0.774038,0.770885,0.976454,1.006071,0.973815,0.769360,0.662910,0.780443
Jesus College,0.708581,0.732202,0.626143,0.680372,0.809099,0.792267,0.790153,0.766786,0.610647,0.676237,0.772395,0.724818
Said Business School,NaN,NaN,0.647588,0.665596,0.977989,0.926032,0.946448,1.044556,0.802452,0.759310,0.712980,0.833606
St Giles,NaN,NaN,0.676957,0.737472,0.842276,0.754305,0.830379,0.954543,0.904060,0.868589,0.724511,0.813531
Ahlul Bayt Centre,NaN,NaN,0.645467,0.750594,0.964230,0.907080,0.961567,0.918494,0.884454,0.836218,0.769215,0.852739
County Hall,NaN,NaN,NaN,NaN,NaN,0.907238,0.964253,0.953771,0.872713,0.819623,0.797349,0.900705
Divinity Rd,NaN,NaN,0.592053,0.691063,0.839848,0.930095,0.842543,0.751057,0.738556,0.667532,0.665561,0.752728
High St,NaN,NaN,NaN,0.745046,0.793766,0.919763,0.838833,0.747370,0.668289,0.566992,0.642623,0.750284


In [ ]:
g

,Event code,Sensor,sensor_pm10225_ratio
0,a,Ahlul Bayt Centre,NaN
10,k,Ahlul Bayt Centre,0.769215
1,b,Ahlul Bayt Centre,NaN
2,c,Ahlul Bayt Centre,0.645467
3,d,Ahlul Bayt Centre,0.750594
4,e,Ahlul Bayt Centre,0.964230
5,f,Ahlul Bayt Centre,0.907080
6,g,Ahlul Bayt Centre,0.961567
7,h,Ahlul Bayt Centre,0.918494
8,i,Ahlul Bayt Centre,0.884454


In [ ]:
tmp = dfratio1[['Event code','Sensor','sensor_pm10225_ratio']]
tmp['order'] = tmp['Sensor'].map(sort_map1)
tmp['order'] = tmp['order'].astype('int32')
tmp.sort_values(by='order',inplace=True)
tmp = tmp.drop(columns=['order'])

tmp

,Event code,Sensor,sensor_pm10225_ratio
76,k,New Marston,NaN
67,b,New Marston,NaN
66,a,New Marston,NaN
68,c,New Marston,0.622611
69,d,New Marston,0.606669
...,...,...,...
159,f,Worcester College,1.020185
160,g,Worcester College,0.843006
161,h,Worcester College,0.708729
162,i,Worcester College,0.660048


In [9]:


#dfratio1x = dfratio1.iloc[:, :-1]
#tmp = dfratio1[['Event code','Sensor','sensor_pm10225_ratio']]
#dfratio1x = dfratio1x.drop(columns=['order'])

#dfratio1['order'] = dfratio1['Sensor'].map(sort_map1)
#dfratio1x = dfratio1.iloc[:, :-1]
#tmp['order'] = dfratio1['order'].astype('int32')
# tmp.sort_values(by='order',inplace=True)
# dfratio1 = dfratio1.drop(columns=['order'])

# tmplist = []
# for i, g in tmp.groupby(['Sensor']):
#     stat, p = wilcoxon(x=g['sensor_pm10225_ratio'].values, y=g['sensor_pm10225_ratio'].values) 
#     tmplist.append([i[0],i[1],stat,p])
# pm25210_ratio_wilcoxon = pd.DataFrame(tmplist, columns = ['Sensor','Event code', 'stat', 'P'])



# dif_pm25_p95 = df[['Event code','Sensor','dif_pm25_95']].pivot(index='Sensor',columns='Event code').round(2).droplevel(0, axis=1)
# dif_pm25_p95.loc['Mean',:] = dif_pm25_p95.mean(axis=0)
# dif_pm25_p95.loc[:,'Mean'] = dif_pm25_p95.mean(axis=1)

# pm25210_ratio_wilcoxon['order'] = pm25210_ratio_wilcoxon['Sensor'].map(sort_map1)
# pm25210_ratio_wilcoxon['order'] = pm25210_ratio_wilcoxon['order'].astype('int32')
# pm25210_ratio_wilcoxon.sort_values(by='order',inplace=True)

# pm25_mean_wilcoxon['order'] = pm25_mean_wilcoxon['fname'].map(sort_map)
# pm25_mean_wilcoxon['order'] = pm25_mean_wilcoxon['order'].astype('int32')
# pm25_mean_wilcoxon.sort_values(by='order',inplace=True)

dfratio1x.loc['Mean',:] = dfratio1x.mean(axis=0)
dfratio1x.loc[:,'Mean'] = dfratio1x.mean(axis=1)
dfratio1x['order'] = dfratio1x.index.map(sort_map1)
dfratio1x['order'] = dfratio1x['order'].astype('int32')
dfratio1x.sort_values(by='order',inplace=True)
dfratio1x = dfratio1x.drop(columns=['order'])

fig, (ax1)  = plt.subplots(1, figsize=(17,9))

sns.heatmap(dfratio1x, annot=True, cmap='coolwarm', robust=True, ax=ax1, linewidths=.5, vmin=0.6, vmax=1.0, cbar_kws={'label': 'Change (ug/m3)'})
# tmpdf = pm25210_ratio_wilcoxon[['order','Event code', 'P'
#                           ]].pivot(index='order',columns='Event code').droplevel([0], axis=1
#                                             ).reset_index(drop=True
#                                                          ).T.reset_index(drop=True
#                                                                         ).T.stack()
# tmpdf = tmpdf[tmpdf > 0.05]
# t = tmpdf.index.values.tolist()
# for i in t:
#     ax1.add_patch(Rectangle((i[1], i[0]), 1, 1, fill=False, edgecolor='black', lw=2, hatch='//'))
 
ax1.set_title('(a)   Daily mean PM2.5 to PM10 ratio  by lockdown event (PM2.5 /PM10)', y=1.01, x=0.4, fontsize=14.5)    
ax1.set_xlabel('Lockdown event code', fontsize=13)
ax1.set_ylabel('Sensor location', fontsize=13)


fig.suptitle('Measured differentials in PM2.5 to PM10 ratio during 2020-21 COVID-19 pandemic restriction events relative to typical levels experienced 2016-19 in Oxford, UK.\nDifferences shown in mass units (ug/m3). All changes statistically significant differences unless denoted by shading.', fontsize=14, y=1)

#plt.tight_layout() 
plt.subplots_adjust(wspace=0.35, hspace=0.2)

#plt.savefig('sensor_vs_typical_year_heatmap_24hrmean_1619.png')

plt.show()

    

NameError: name 'dfratio1x' is not defined

In [ ]:
dif_dfratio1

Event code,a,b,c,d,e,f,g,h,i,j,k,Mean,order
Sensor,,,,,,,,,,,,,
Divinity Rd,NaN,NaN,0.592053,0.691063,0.839848,0.930095,0.842543,0.751057,0.738556,0.667532,0.665561,0.746479,0
High St,NaN,NaN,NaN,0.745046,0.793766,0.919763,0.838833,0.747370,0.668289,0.566992,0.642623,0.740335,1
South Parks Rd,NaN,NaN,0.565735,0.594352,0.876873,0.876233,0.846483,0.749038,0.692100,0.614434,0.646783,0.718003,2
Worcester College,0.762535,0.692612,0.788304,0.864375,0.978684,1.020185,0.843006,0.708729,0.660048,0.626271,0.798592,0.794849,3
Ahlul Bayt Centre,NaN,NaN,0.645467,0.750594,0.964230,0.907080,0.961567,0.918494,0.884454,0.836218,0.769215,0.848591,4
County Hall,NaN,NaN,NaN,NaN,NaN,0.907238,0.964253,0.953771,0.872713,0.819623,0.797349,0.885824,5
Jesus College,0.708581,0.732202,0.626143,0.680372,0.809099,0.792267,0.790153,0.766786,0.610647,0.676237,0.772395,0.724080,6
John Radcliffe,NaN,NaN,NaN,NaN,NaN,0.763753,0.808919,0.690403,0.625035,0.509233,0.587016,0.664060,7
Said Business School,NaN,NaN,0.647588,0.665596,0.977989,0.926032,0.946448,1.044556,0.802452,0.759310,0.712980,0.831439,8
